# A Novel Embedding Model for Knowledge Base Completion Based on Convolutional Neural Network

## Abstract
Trong bài báo này, chúng tôi đề xuất một mô hình nhúng mới, có tên ConvKB, để hoàn thành cơ sở tri thức. Mô hình ConvKB của chúng tôi cải tiến các mô hình tiên tiến bằng cách sử dụng mạng thần kinh tích chập, để nó có thể nắm bắt các mối quan hệ toàn cầu và các đặc điểm chuyển tiếp giữa các thực thể và quan hệ trong các cơ sở tri thức. Trong ConvKB, mỗi bộ ba (thực thể đầu, quan hệ, thực thể đuôi) được biểu diễn dưới dạng ma trận 3 cột trong đó mỗi vectơ cột biểu thị một phần tử ba. Ma trận 3 cột này sau đó được đưa đến một lớp chập trong đó nhiều bộ lọc được vận hành trên ma trận để tạo ra các bản đồ tính năng khác nhau. Các bản đồ tính năng này sau đó được nối vào một vectơ tính năng duy nhất đại diện cho bộ ba đầu vào. Vectơ đặc trưng được nhân với một vectơ trọng lượng thông qua một sản phẩm chấm để trả về điểm số. Điểm này sau đó được sử dụng để dự đoán liệu bộ ba có hợp lệ hay không. Các thử nghiệm cho thấy ConvKB đạt được hiệu suất dự đoán liên kết tốt hơn so với các mô hình nhúng tiên tiến trước đây trên hai bộ dữ liệu điểm chuẩn WN18RR và FB15k-237